# Extracting Data From the WMO app

## 1 Insturactions of the WMO

Use the following steps to carry out the project
- Use this link (https://community.wmo.int/en/members/profiles to extract data on
heatwaves warning system/services, see example below, if heatwave is in the list of
warnings provided, then indicate a yes in the provided excel sheet (for the corresponding
country in column B in the excel sheet called “Heatwaves Warning Systems (WMO data)”).


full screenmode : https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9


## 2  Required Libraries

In [673]:
#The webdriver module in Selenium allows for automated control of web browsers.
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


## 3 Opening up the web browser and printing a sample text

The function below opens up chrome browser and waits for 60 minutes to load the data 

input: The website link

Output : the driver and wabdriverwait instance

In [674]:

def openChrome(link):
    # Options customize Chrome WebDriver behavior
    options = Options()

    # Initialize the WebDriver (e.g., Chrome)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Navigate to the desired URL
    driver.get(link)

    # wait for page to load
    wait = WebDriverWait(driver, 60)

    return driver, wait

In [675]:
link = "https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9"
driver_instance, wait  = openChrome(link)

Using the driver instance that we created print an sample text

In [570]:
# Locate all div elements with class "textbox"
div_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="textbox"]')))

# Iterate over the list of elements and print the text content of each
for div_element in div_elements:
    print(div_element.text)
    print("-----")  # Just a separator for clarity

# driver_instance.quit()

Member Profiles
-----
The information contained in these profiles have been provided by WMO Members. The information is based on a self-assessment exercise. The date displayed in each section corresponds with the date in which the information was provided to WMO. 

If you find any issue or inconsistency with the information displayed, please contact community@wmo.int.
-----
Permanent Representative:
-----
Hydrological Adviser:
-----
National Meteorological Service:
-----
Responsible Ministry:
-----


## 4. Clicking on the Early Warning Services 

In [571]:
#EC stands for "Expected Conditions." It is a set of predefined conditions that the Selenium WebDriver uses.
#presence_of_element_located is one of many expected conditions (EC). It checks if an element is present on the DOM of a page.
def wait_and_click(driver, wait, css_selector):
    element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
    driver.execute_script("arguments[0].click();", element)

In [676]:
css_selector = 'a[title=" Early Warning Services"]'
wait_and_click(driver_instance, wait, css_selector)

## 5 - Get Warnings List

In [677]:
def saveWarnings(driver, css_selector):
    #'.title' is the class name of the element that contains the warning text
    warnings= driver_instance.find_elements(By.CSS_SELECTOR, css_selector)
    return warnings

warnings = saveWarnings(driver_instance, '.title')


In [678]:

for warning in warnings:
    print(warning.text)

Dust storm/Sandstorm
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
Yes
2 jours
Yes
Yes
No
Yes
Yes


## 6 getting all country names 

In [575]:

def getCountryNames(driver, wait):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    loaded_countries = set()
    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Scroll to the last element to load more countries
        ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()
    print(loaded_countries)
    # Close the dropdown menu by clicking the dropdown button again
    dropdown.click()
    return loaded_countries

In [679]:
loaded_countries = getCountryNames(driver_instance, wait)

{'Solomon Islands', 'Ireland', 'Romania', 'Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco', 'United States of America', 'Cyprus', 'Andorra', 'Italy', 'Lithuania', 'Guatemala', 'Thailand', 'Fiji', 'Venezuela, Bolivarian Republic of', 'Eswatini', "Lao People's Democratic Republic", 'Belarus', 'Guinea', 'Latvia', 'India', 'Portugal', 'Qatar', 'Republic of Korea', 'Bulgaria', 'North Macedonia', 'Slovakia', 'United Republic of Tanzania', 'Iceland', 'Montenegro', 'Uganda', 'Austria', 'Mali', 'Armenia', 'Cameroon', 'Greece', 'Costa Rica', 'Israel', 'Bahrain', 'Sweden', 'Liberia', 'Afghanistan', 'Myanmar', 'Nicaragua', 'Tuvalu', 'Saudi Arabia', 'Lebanon', 'Cambodia', 'Ghana', 'Botswana', 'British Caribbean Territories - Anguilla', 'Congo', 'Germany', 'Antigua and Barbuda', 'British Caribbean Territories - Montserrat', 'Brazil', 'New Zealand', 'Czechia', 'Cabo Verde', 'Micronesia (Federated States of)', 'Central African Republic', 'French Polynesia', 'Angola', 'Cuba', 'Hondur

In [680]:
print(len(loaded_countries ))

198


## 7 Selecting A country

In [681]:
def select_country_from_dropdown(driver, wait, target_country):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    # Clear the search input
    search_input = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.searchInput'))
    )
    search_input.clear()

    loaded_countries = set()
    country_selected = False

    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if target country is in the current batch of loaded countries
        if target_country in current_countries:
            for option in current_countries_elements:
                if option.text == target_country:
                    option.click()
                    country_selected = True
                    break

        # If the country has been selected, break out of the loop
        if country_selected:
            break
        
        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Scroll to the last element to load more countries
        ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()

    return country_selected


In [682]:
success = select_country_from_dropdown(driver_instance, wait, "United States of America")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

has been selected successfully!


In [683]:
success = select_country_from_dropdown(driver_instance, wait, "India")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

has been selected successfully!


In [614]:

warnings = saveWarnings(driver_instance, '.title')

for warning in warnings:
    print(warning.text)

Dust storm/Sandstorm
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
Yes
2 jours
Yes
Yes
No
Yes
Yes


In [684]:
success = select_country_from_dropdown(driver_instance, wait, "Argentina")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

has been selected successfully!


In [604]:
warnings = saveWarnings(driver_instance, '.title')

for warning in warnings:
    print(warning.text)

Cold wave
Drought/Dry spell
Fog
Freezing rain
Frost
Hail
Haze/Smoke
Heat wave
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Volcanic ash
Wind
Yes
Yes
Yes
máximo 72 hs
Yes
Yes
No
Yes
No
el 100


## selecting 8 country and getting the information

1- Initiate global variables

    - initiate a dictinary

    - driver, wait

2- Load all the country names

3 -Click on Early warning systems

----- repeat this 8 times ----

4- Click on the desired country name

5- extract warning systems in a list

   (if the list is empty)
   
6- save country name and the list in the map

In [ ]:
# Convert the set to a list
countries_list = list(loaded_countries)

# Sort the list
countries_list.sort()

print(countries_list)
test= countries_list[0:8]

In [670]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

def select_country_from_dropdown(driver, wait, target_country):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    # Clear the search input
    search_input = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.searchInput'))
    )
    search_input.clear()

    loaded_countries = set()
    country_selected = False

    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if target country is in the current batch of loaded countries
        if target_country in current_countries:
            for option in current_countries_elements:
                if option.text == target_country:
                    option.click()
                    country_selected = True
                    break

        # If the country has been selected, break out of the loop
        if country_selected:
            break
        
        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Scroll using JavaScript
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", current_countries_elements[-1])
        time.sleep(1)  # Give some time for the new countries to load

    return country_selected


In [685]:
test_list = ['Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco']
print(len(test_list))
my_dict = {}

4


In [686]:
for country in test_list:
    print(country)
    time.sleep(5)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(5)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(10)
        


Guyana
country been selected successfully!
Drought/Dry spell
Flood
Fog
Heat wave
High Seas/Rogue waves etc.
Landslide/Mudslide & Debris flow
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Wind
Yes
Yes
Yes
No
Yes
Yes
No
No
No data
Drought/Dry spell
Flood
Fog
Heat wave
High Seas/Rogue waves etc.
Landslide/Mudslide & Debris flow
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Wind
Yes
Yes
Yes
No
Yes
Yes
No
No
No data
-----
Turkmenistan
country been selected successfully!
No warnings found for this country.
-----
British Caribbean Territories
country been selected successfully!
Drought/Dry spell
Flood
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Tropical cyclone
Tsunami
Wind
No
Yes
No
Yes
72 hrs
No
Yes
No
Yes
No
Yes
No
Yes
Yes
100
Drought/Dry spell
Flood
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Tropical cyclone
Tsunami
Wind
No
Yes
No
Yes
72 hrs
No
Yes
No
Yes
No

In [687]:
my_dict

{'Guyana': ['Drought/Dry spell',
  'Flood',
  'Fog',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'Landslide/Mudslide & Debris flow',
  'Lightning',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Thunderstorms/Squall lines',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'No',
  'No',
  'No data'],
 'Turkmenistan': [0],
 'British Caribbean Territories': ['Drought/Dry spell',
  'Flood',
  'Lightning',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Thunderstorms/Squall lines',
  'Tropical cyclone',
  'Tsunami',
  'Wind',
  'No',
  'Yes',
  'No',
  'Yes',
  '72 hrs',
  'No',
  'Yes',
  'No',
  'Yes',
  'No',
  'Yes',
  'No',
  'Yes',
  'Yes',
  '100'],
 'Monaco': [0]}

In [688]:
test_list = ['Solomon Islands', 'Ireland', 'Romania', 'Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco', 'United States of America']
print(len(test_list))
my_dict = {}

8


In [689]:
for country in test_list:
    print(country)
    time.sleep(5)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(5)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(10)



Solomon Islands
country been selected successfully!
No warnings found for this country.
-----
Ireland
country been selected successfully!
Cold wave
Fog
Freezing rain
Hail
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wind
Yes
Yes
Yes
2-days
Yes
Yes
No
No
100000
Cold wave
Fog
Freezing rain
Hail
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wind
Yes
Yes
Yes
2-days
Yes
Yes
No
No
100000
-----
Romania
country been selected successfully!
Avalanche
Cold wave
Drought/Dry spell
Extra-tropical cyclone
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Storm surge/Coastal flood
Yes
Yes
Yes
Up to 3…5 days for general warnings and up to 6 hours for nowcasting warnings
Yes
Yes
Yes
No
Emergency Mobile Alerts (EMA) is an alerting network in Romania designed to disseminate emergency alerts to mobile devices.

10 country

In [704]:
test_list = [ 'Solomon Islands', 'Ireland', 'Romania', 'Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco', 'United States of America', 'Cyprus']
print(len(test_list))
my_dict = {}

9


In [705]:
for country in test_list:
    print(country)
    time.sleep(12)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(5)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(10)

Solomon Islands
country been selected successfully!
No warnings found for this country.
-----
Ireland
country been selected successfully!
Cold wave
Fog
Freezing rain
Hail
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wind
Yes
Yes
Yes
2-days
Yes
Yes
No
No
100000
Cold wave
Fog
Freezing rain
Hail
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wind
Yes
Yes
Yes
2-days
Yes
Yes
No
No
100000
-----
Romania
country been selected successfully!
Avalanche
Cold wave
Drought/Dry spell
Extra-tropical cyclone
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Storm surge/Coastal flood
Yes
Yes
Yes
Up to 3…5 days for general warnings and up to 6 hours for nowcasting warnings
Yes
Yes
Yes
No
Emergency Mobile Alerts (EMA) is an alerting network in Romania designed to disseminate emergency alerts to mobile devices.

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=117.0.5938.88)
Stacktrace:
0   chromedriver                        0x0000000109f77e58 chromedriver + 5090904
1   chromedriver                        0x0000000109f6ebc3 chromedriver + 5053379
2   chromedriver                        0x0000000109b0a366 chromedriver + 447334
3   chromedriver                        0x0000000109b4efd1 chromedriver + 729041
4   chromedriver                        0x0000000109b80662 chromedriver + 931426
5   chromedriver                        0x0000000109b4cba8 chromedriver + 719784
6   chromedriver                        0x0000000109b8081e chromedriver + 931870
7   chromedriver                        0x0000000109b9c6e1 chromedriver + 1046241
8   chromedriver                        0x0000000109b80433 chromedriver + 930867
9   chromedriver                        0x0000000109b4b042 chromedriver + 712770
10  chromedriver                        0x0000000109b4c26e chromedriv

## 9 adjusting so we can run the full loop

In [671]:
test_list = ['Solomon Islands', 'Ireland', 'Romania', 'Guyana']
print(len(test_list))
my_dict = {}

4


In [672]:
success = select_country_from_dropdown(driver_instance, wait, "Benin")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

Failed to select!


In [ ]:
success = select_country_from_dropdown(driver_instance, wait, country)

In [663]:
for country in test_list:
    print(country)
    success = select_country_from_dropdown(driver_instance, wait, country)
    if success:
        print("country been selected successfully!")
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
    else:
        print("Failed to select the country!")
        


Solomon Islands
Failed to select the country!
Ireland


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=117.0.5938.88)
Stacktrace:
0   chromedriver                        0x000000010df7ae58 chromedriver + 5090904
1   chromedriver                        0x000000010df71bc3 chromedriver + 5053379
2   chromedriver                        0x000000010db0d366 chromedriver + 447334
3   chromedriver                        0x000000010db51fd1 chromedriver + 729041
4   chromedriver                        0x000000010db83662 chromedriver + 931426
5   chromedriver                        0x000000010db4fba8 chromedriver + 719784
6   chromedriver                        0x000000010db8381e chromedriver + 931870
7   chromedriver                        0x000000010db9f6e1 chromedriver + 1046241
8   chromedriver                        0x000000010db83433 chromedriver + 930867
9   chromedriver                        0x000000010db4e042 chromedriver + 712770
10  chromedriver                        0x000000010db4f26e chromedriv